# AIPI510 Fall 2024 Assignment # 2

### Team Details

**Team name:** Forever Loop

**Team members:** Santosh Ganesan, Haran Nallasivan

### Reference Used
We used [this Medium article](https://medium.com/@BrandonSouthern/sql-best-practices-e1c61e96ee27) for a set of best practices for SQL formatting.

### CREATE
You have been provided with a sample .csv file. Create a SQLite database in Python using the data in this sample .csv file.

#### Import statements and shared functions

In [21]:
import pandas as pd
import sqlite3

def query(sql, args=None):
    """
    Utility function to factor out duplicative code for connecting to the database, creating a cursor,
    executing a query, committing on the connection, and fetching results from the cursor. Uses a context manager,
    parameterized queries, and a try-except block to follow best practices.

    Parameters:
        sql (str) - SQL query expression to be executed on the tips table in the tips.db SQLite database.
        args (dictionary, optional) - Arguments to be passed if the query is parameterized.
    """
    try:
        with sqlite3.connect('./tips.db') as conn:
            c = conn.cursor()
            if args is not None:
                c.execute(sql, args)
            else:
                c.execute(sql)
            conn.commit()
            return c.fetchall()
    except:
        print(f'Failed to execute query:\n{sql}')
        return []

#### Exploratory analysis of the data

In [22]:

df = pd.read_csv('data/tips.csv')
print(df.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


#### Create the SQLite database

In [23]:

with sqlite3.connect('tips.db') as conn:
    c = conn.cursor()
    c.execute('''CREATE TABLE tips (total_bill,tip,sex,smoker,day,time,size)''')
    conn.commit()

#### Verify connectivity to the database

In [24]:
query('''SELECT * from tips;''')

[]

#### Load data into the database

In [25]:
with sqlite3.connect('tips.db') as conn:
    df = pd.read_csv('data/tips.csv')
    df.to_sql('tips', conn, if_exists='replace', index=False)
    conn.commit()


### READ
Answer the following questions (must show your work via SQL queries):
1. Retrieve the average tip percentage for each day of the week 


In [26]:
query('''
      SELECT
            ROUND(AVG(tip/total_bill) * 100) AS average_tip_percentage
      FROM
            tips;
      ''')

[(16.0,)]

2. Find the maximum and minimum total bull amounts


In [27]:
query('''
      SELECT
            MAX(total_bill) AS max,
            MIN(total_bill) AS min
      FROM
            tips;
      ''')

[(50.81, 3.07)]

3. Count the number of parties for each size


In [28]:
query('''
      SELECT
            size,
            COUNT(*) as count
      FROM
            tips
      GROUP BY
            size;
      ''')

[(1, 4), (2, 156), (3, 38), (4, 37), (5, 5), (6, 4)]

4. Retrieve the total bill and tip for parties of size 4 or more, where the tip percentage is greater than 15%


In [29]:
query('''
      SELECT
            total_bill,
            tip,
            ROUND(tip/total_bill * 100) AS tip_percentage
      FROM
            tips
      WHERE
            size > :size 
            AND tip_percentage > :tip_percentage;
      ''', { 'size': 4, 'tip_percentage': 15 })

[(34.3, 6.7, 20.0),
 (27.05, 5.0, 18.0),
 (29.85, 5.14, 17.0),
 (20.69, 5.0, 24.0)]


5. Retrieve the total bill, tip amount, and tip percentage for each combination of day and time, sorted by tip percentage in descending order

In [30]:
query('''
      SELECT
            day,
            time,
            total_bill,
            tip,
            ROUND((tip/total_bill) * 100) AS tip_percentage
      FROM
            tips
      GROUP BY 
            day,
            time
      ORDER BY 
            tip_percentage
      DESC;
      ''')

[('Fri', 'Lunch', 12.16, 2.2, 18.0),
 ('Sat', 'Dinner', 20.65, 3.35, 16.0),
 ('Thur', 'Dinner', 18.78, 3.0, 16.0),
 ('Thur', 'Lunch', 27.2, 4.0, 15.0),
 ('Fri', 'Dinner', 28.97, 3.0, 10.0),
 ('Sun', 'Dinner', 16.99, 1.01, 6.0)]

6. Find the average tip percentage for each combination of day, time, and smoker status


In [31]:
query('''
      SELECT
            ROUND(AVG(tip/total_bill)*100),
            day,
            smoker,
            time
      FROM
            tips
      GROUP BY
            day,
            smoker,
            time;
      ''')  

[(14.0, 'Fri', 'No', 'Dinner'),
 (19.0, 'Fri', 'No', 'Lunch'),
 (17.0, 'Fri', 'Yes', 'Dinner'),
 (19.0, 'Fri', 'Yes', 'Lunch'),
 (16.0, 'Sat', 'No', 'Dinner'),
 (15.0, 'Sat', 'Yes', 'Dinner'),
 (16.0, 'Sun', 'No', 'Dinner'),
 (19.0, 'Sun', 'Yes', 'Dinner'),
 (16.0, 'Thur', 'No', 'Dinner'),
 (16.0, 'Thur', 'No', 'Lunch'),
 (16.0, 'Thur', 'Yes', 'Lunch')]

7. Retrieve the total bill, tip amount, and tip percentage for each sex, sorted by total bill in descending order, and limit the results to the top 5 records


In [33]:
query('''
      SELECT
            ROUND(SUM(total_bill), 2),
            ROUND(SUM(tip), 2),
            ROUND(SUM(tip)/SUM(total_bill)*100) AS tip_percentage,
            sex
      FROM
            tips
      GROUP BY
            sex
      ORDER BY
            total_bill
      DESC;
      ''') 

[(1570.95, 246.51, 16.0, 'Female'), (3256.82, 485.07, 15.0, 'Male')]

8. Find the maximum and minimum tip percentage for each day and time combination, along with the corresponding total bill and tip amount

In [34]:
query('''
      SELECT
            ROUND(MAX(tip/total_bill)*100) AS max_tip_percentage,
            ROUND(MIN(tip/total_bill)*100) AS min_tip_percentage,
            day,
            time,
            total_bill,
            tip
      FROM
            tips
      GROUP BY
            day,
            time;
      ''')

[(26.0, 10.0, 'Fri', 'Dinner', 28.97, 3.0),
 (26.0, 12.0, 'Fri', 'Lunch', 13.42, 1.58),
 (33.0, 4.0, 'Sat', 'Dinner', 32.83, 1.17),
 (71.0, 6.0, 'Sun', 'Dinner', 16.99, 1.01),
 (16.0, 16.0, 'Thur', 'Dinner', 18.78, 3.0),
 (27.0, 7.0, 'Thur', 'Lunch', 18.64, 1.36)]

9. Retrieve the total bill, tip amount, and tip percentage for parties of size 4 or more, where the tip percentage is greater than 15%, and the total bill is between $50 and $100


In [35]:
query('''
      SELECT
            total_bill,
            tip,
            ROUND((tip/total_bill)*100) AS tip_percentage,
            size
      FROM
            tips
      WHERE
            size >= :size
      AND
            tip_percentage > :tip_percentage
      AND
            total_bill BETWEEN :total_bill_min AND :total_bill_max;
      ''', { 'size': 4, 'tip_percentage': 15, 'total_bill_min': 50, 'total_bill_max': 100 })

[]

10. Find the average tip percentage for each combination of day, time, and smoker status, but only include combinations with more than 5 records


In [36]:
query('''
      SELECT
            ROUND(AVG(tip/total_bill)*100),
            day,
            time,
            smoker,
            COUNT(*)
      FROM
            tips
      GROUP BY
            day,
            time,
            smoker
      HAVING 
            COUNT(*) > :count;
      ''', { 'count': 5 })

[(17.0, 'Fri', 'Dinner', 'Yes', 9),
 (19.0, 'Fri', 'Lunch', 'Yes', 6),
 (16.0, 'Sat', 'Dinner', 'No', 45),
 (15.0, 'Sat', 'Dinner', 'Yes', 42),
 (16.0, 'Sun', 'Dinner', 'No', 57),
 (19.0, 'Sun', 'Dinner', 'Yes', 19),
 (16.0, 'Thur', 'Lunch', 'No', 44),
 (16.0, 'Thur', 'Lunch', 'Yes', 17)]

Come up with your own SQL queries for the table. You must show 5 additional queries beyond the 10 outlined above.


Supplemental 1. Retrieve the total bill, tip, and size and find the difference of the total bill from the average total bill.


In [37]:
query('''
      SELECT
            total_bill,
            tip,
            average_total_bill,
            ROUND(total_bill - average_total_bill, 2) AS difference,
            size
      FROM
            tips,
            (SELECT ROUND(AVG(total_bill), 2) as average_total_bill FROM tips) AS averages_table;
      ''')

[(16.99, 1.01, 19.79, -2.8, 2),
 (10.34, 1.66, 19.79, -9.45, 3),
 (21.01, 3.5, 19.79, 1.22, 3),
 (23.68, 3.31, 19.79, 3.89, 2),
 (24.59, 3.61, 19.79, 4.8, 4),
 (25.29, 4.71, 19.79, 5.5, 4),
 (8.77, 2.0, 19.79, -11.02, 2),
 (26.88, 3.12, 19.79, 7.09, 4),
 (15.04, 1.96, 19.79, -4.75, 2),
 (14.78, 3.23, 19.79, -5.01, 2),
 (10.27, 1.71, 19.79, -9.52, 2),
 (35.26, 5.0, 19.79, 15.47, 4),
 (15.42, 1.57, 19.79, -4.37, 2),
 (18.43, 3.0, 19.79, -1.36, 4),
 (14.83, 3.02, 19.79, -4.96, 2),
 (21.58, 3.92, 19.79, 1.79, 2),
 (10.33, 1.67, 19.79, -9.46, 3),
 (16.29, 3.71, 19.79, -3.5, 3),
 (16.97, 3.5, 19.79, -2.82, 3),
 (20.65, 3.35, 19.79, 0.86, 3),
 (17.92, 4.08, 19.79, -1.87, 2),
 (20.29, 2.75, 19.79, 0.5, 2),
 (15.77, 2.23, 19.79, -4.02, 2),
 (39.42, 7.58, 19.79, 19.63, 4),
 (19.82, 3.18, 19.79, 0.03, 2),
 (17.81, 2.34, 19.79, -1.98, 4),
 (13.37, 2.0, 19.79, -6.42, 2),
 (12.69, 2.0, 19.79, -7.1, 2),
 (21.7, 4.3, 19.79, 1.91, 2),
 (19.65, 3.0, 19.79, -0.14, 2),
 (9.55, 1.45, 19.79, -10.24, 2),
 (1

Supplemental 2. Retrieve the total bill, smoker, and day for parties of size 2 or 3.


In [38]:
query('''
      SELECT
            total_bill,
            smoker,
            day,
            size
      FROM
            tips
      WHERE
            size IN(:size1, :size2);
      ''', { 'size1': 2, 'size2': 3 })

[(16.99, 'No', 'Sun', 2),
 (10.34, 'No', 'Sun', 3),
 (21.01, 'No', 'Sun', 3),
 (23.68, 'No', 'Sun', 2),
 (8.77, 'No', 'Sun', 2),
 (15.04, 'No', 'Sun', 2),
 (14.78, 'No', 'Sun', 2),
 (10.27, 'No', 'Sun', 2),
 (15.42, 'No', 'Sun', 2),
 (14.83, 'No', 'Sun', 2),
 (21.58, 'No', 'Sun', 2),
 (10.33, 'No', 'Sun', 3),
 (16.29, 'No', 'Sun', 3),
 (16.97, 'No', 'Sun', 3),
 (20.65, 'No', 'Sat', 3),
 (17.92, 'No', 'Sat', 2),
 (20.29, 'No', 'Sat', 2),
 (15.77, 'No', 'Sat', 2),
 (19.82, 'No', 'Sat', 2),
 (13.37, 'No', 'Sat', 2),
 (12.69, 'No', 'Sat', 2),
 (21.7, 'No', 'Sat', 2),
 (19.65, 'No', 'Sat', 2),
 (9.55, 'No', 'Sat', 2),
 (15.06, 'No', 'Sat', 2),
 (17.78, 'No', 'Sat', 2),
 (24.06, 'No', 'Sat', 3),
 (16.31, 'No', 'Sat', 3),
 (16.93, 'No', 'Sat', 3),
 (18.69, 'No', 'Sat', 3),
 (31.27, 'No', 'Sat', 3),
 (16.04, 'No', 'Sat', 3),
 (17.46, 'No', 'Sun', 2),
 (13.94, 'No', 'Sun', 2),
 (9.68, 'No', 'Sun', 2),
 (18.29, 'No', 'Sun', 2),
 (22.23, 'No', 'Sun', 2),
 (28.55, 'No', 'Sun', 3),
 (18.04, 'No', '

Supplemental 3. Segregate all smokers as sitting outside while non smokers sit inside.


In [39]:
query('''
      SELECT
            smoker,
            day,
            time,
            CASE smoker
                WHEN 'No' THEN 'Inside'
                ELSE 'Outside'
            END setting
      FROM
            tips;
      ''')

[('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sun', 'Dinner', 'Inside'),
 ('No', 'Sat', 'Dinner', 'Inside'),
 ('No', 'Sat', 'Dinner', 'Inside'),
 ('No', 'Sat', 'Dinner', 'Inside'),
 ('No', 'Sat', 'Dinner', 'Inside'),
 ('No', 'Sat', 'Dinner', 'Inside'),
 ('No', 'Sat', 'Dinner', 'Inside'),
 ('No', 'Sat', 'Dinner', 'Inside'),
 ('No', 'Sat', 'Dinner', 'Inside'),
 ('No', 'Sat', 'Dinner', 'In

Supplemental 4. Retrieve the following 20 entries for total bill, tip, sex, smoker, day, time, and size from the 15th entry in the table.


In [40]:
query('''
      SELECT
            total_bill,
            tip,
            sex,
            smoker,
            day,
            time,
            size
      FROM
            tips
      LIMIT 
            :page_size
      OFFSET
            :offset;
      ''', { 'page_size': 20, 'offset': 14 })

[(14.83, 3.02, 'Female', 'No', 'Sun', 'Dinner', 2),
 (21.58, 3.92, 'Male', 'No', 'Sun', 'Dinner', 2),
 (10.33, 1.67, 'Female', 'No', 'Sun', 'Dinner', 3),
 (16.29, 3.71, 'Male', 'No', 'Sun', 'Dinner', 3),
 (16.97, 3.5, 'Female', 'No', 'Sun', 'Dinner', 3),
 (20.65, 3.35, 'Male', 'No', 'Sat', 'Dinner', 3),
 (17.92, 4.08, 'Male', 'No', 'Sat', 'Dinner', 2),
 (20.29, 2.75, 'Female', 'No', 'Sat', 'Dinner', 2),
 (15.77, 2.23, 'Female', 'No', 'Sat', 'Dinner', 2),
 (39.42, 7.58, 'Male', 'No', 'Sat', 'Dinner', 4),
 (19.82, 3.18, 'Male', 'No', 'Sat', 'Dinner', 2),
 (17.81, 2.34, 'Male', 'No', 'Sat', 'Dinner', 4),
 (13.37, 2.0, 'Male', 'No', 'Sat', 'Dinner', 2),
 (12.69, 2.0, 'Male', 'No', 'Sat', 'Dinner', 2),
 (21.7, 4.3, 'Male', 'No', 'Sat', 'Dinner', 2),
 (19.65, 3.0, 'Female', 'No', 'Sat', 'Dinner', 2),
 (9.55, 1.45, 'Male', 'No', 'Sat', 'Dinner', 2),
 (18.35, 2.5, 'Male', 'No', 'Sat', 'Dinner', 4),
 (15.06, 3.0, 'Female', 'No', 'Sat', 'Dinner', 2),
 (20.69, 2.45, 'Female', 'No', 'Sat', 'Dinner

Supplemental 5. Retrieve the total bill, tips, and day for all smokers.


In [41]:
query('''
      SELECT 
            total_bill,
            tip,
            sex,
            smoker,
            day,
            time,
            size
      FROM
            tips
      WHERE
            smoker LIKE '%yes%';
      ''')

[(38.01, 3.0, 'Male', 'Yes', 'Sat', 'Dinner', 4),
 (11.24, 1.76, 'Male', 'Yes', 'Sat', 'Dinner', 2),
 (20.29, 3.21, 'Male', 'Yes', 'Sat', 'Dinner', 2),
 (13.81, 2.0, 'Male', 'Yes', 'Sat', 'Dinner', 2),
 (11.02, 1.98, 'Male', 'Yes', 'Sat', 'Dinner', 2),
 (18.29, 3.76, 'Male', 'Yes', 'Sat', 'Dinner', 4),
 (3.07, 1.0, 'Female', 'Yes', 'Sat', 'Dinner', 1),
 (15.01, 2.09, 'Male', 'Yes', 'Sat', 'Dinner', 2),
 (26.86, 3.14, 'Female', 'Yes', 'Sat', 'Dinner', 2),
 (25.28, 5.0, 'Female', 'Yes', 'Sat', 'Dinner', 2),
 (17.92, 3.08, 'Male', 'Yes', 'Sat', 'Dinner', 2),
 (19.44, 3.0, 'Male', 'Yes', 'Thur', 'Lunch', 2),
 (32.68, 5.0, 'Male', 'Yes', 'Thur', 'Lunch', 2),
 (28.97, 3.0, 'Male', 'Yes', 'Fri', 'Dinner', 2),
 (5.75, 1.0, 'Female', 'Yes', 'Fri', 'Dinner', 2),
 (16.32, 4.3, 'Female', 'Yes', 'Fri', 'Dinner', 2),
 (40.17, 4.73, 'Male', 'Yes', 'Fri', 'Dinner', 4),
 (27.28, 4.0, 'Male', 'Yes', 'Fri', 'Dinner', 2),
 (12.03, 1.5, 'Male', 'Yes', 'Fri', 'Dinner', 2),
 (21.01, 3.0, 'Male', 'Yes', 'Fri'

**After you have performed the above queries:**

### UPDATE
It was determined that there was an error in the database. Please update the record that corresponds to id=10 and set smoker to Yes. 


In [42]:
with sqlite3.connect('tips.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT
                    *
              FROM
                    tips
              WHERE
                    rowid = :rowid;
              ''', { 'rowid': 10 })
    conn.commit()
    print(f'Before:\n{c.fetchall()}')
    c.execute('''
              UPDATE
                    tips
              SET
                    smoker = 'Yes'
              WHERE
                    rowid = :rowid;
              ''', { 'rowid': 10 })
    conn.commit()
    c.execute('''
              SELECT
                    *
              FROM
                    tips
              WHERE
                    rowid = :rowid;
              ''', { 'rowid': 10 })
    conn.commit()
    print(f'After:\n{c.fetchall()}')

Before:
[(14.78, 3.23, 'Male', 'No', 'Sun', 'Dinner', 2)]
After:
[(14.78, 3.23, 'Male', 'Yes', 'Sun', 'Dinner', 2)]




### DELETE
Delete records from the database that have a total bill that is less than $10.

In [43]:
with sqlite3.connect('tips.db') as conn:
    c = conn.cursor()
    c.execute('''
              SELECT
                    COUNT(*)
              FROM
                    tips
              WHERE
                    total_bill < :total_bill_threshold;
              ''', { 'total_bill_threshold': 10 })
    conn.commit()
    print(f'Number of rows before: {c.fetchall()[0][0]}')
    c.execute('''
              DELETE FROM
                    tips
              WHERE
                    total_bill < :total_bill_threshold;
              ''', { 'total_bill_threshold': 10 })
    conn.commit()
    c.execute('''
              SELECT
                    COUNT(*)
              FROM
                    tips
              WHERE
                    total_bill < :total_bill_threshold;
              ''', { 'total_bill_threshold': 10 })
    conn.commit()
    print(f'Number of rows before: {c.fetchall()[0][0]}')

Number of rows before: 17
Number of rows before: 0
